# Project Goal: The GitHub Event Lakehouse (Proof of Concept)
The goal of this project is to build a scalable Medallion Architecture that processes high-volume event data from the GitHub Archive. This specific implementation serves as a functional Proof of Concept (PoC), successfully ingesting and transforming data for a targeted window (January 1, 2024, at 15:00 UTC) to validate the end-to-end Spark transformation logic.

The business objective is to identify Developer Velocity—measuring which open-source repositories are receiving the highest density of active code contributions (commits).

## Step-by-Step Technical Implementation
### Source Data Acquisition:

Developed a Python-based ingestion module to programmatically fetch the specific compressed .json.gz archive for the targeted hour.

Validated the integrity of the raw JSON file before moving to the processing phase.

### Bronze Layer (Schema-on-Read & Ingestion):

Utilized Apache Spark to ingest the raw JSON using a manually defined schema (StructType). This is a professional safeguard that prevents pipeline failures due to unexpected changes in the GitHub API data structure.

Appended technical metadata, including _ingestion_timestamp and _source_file, to ensure full data auditability.

Stored the result in Parquet format for optimized disk storage and read efficiency.

### Silver Layer (Data Flattening & Cleaning):

Filtered the dataset specifically for PushEvent types to isolate code-related activities.

Applied Spark’s explode function to transform nested arrays of commits into individual rows.

Partitioned by Date: Structured the storage by event_date and used Dynamic Partition Overwrite. This ensures that if the data for this specific hour is re-processed, only that day’s records are updated, protecting the integrity of the rest of the lakehouse.

### Gold Layer (Business Analytics):

Aggregated the flattened commit data to calculate the Daily Repository Leaderboard.

Calculated metrics for Total Commits and Unique Contributors per repository.

Finalized the analysis into a consumption-ready Parquet table designed for high-speed dashboarding and visualization.



In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from dotenv import load_dotenv

load_dotenv()

spark = SparkSession.builder \
    .appName("GitHub_Lakehouse_Exploration") \
    .getOrCreate()

# Use the paths from your .env
raw_path = os.getenv("DATA_RAW_PATH", "/opt/spark/data/raw/2024-01-01-15.json.gz")
bronze_path = os.getenv("DATA_BRONZE_PATH", "/opt/spark/data/bronze/events")
silver_path = os.getenv("DATA_SILVER_PATH", "/opt/spark/data/silver/commits")
gold_path = os.getenv("DATA_GOLD_PATH", "/opt/spark/data/gold/daily_repo_metrics")

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/28 23:25:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Raw Data: Initial Ingestion

In [4]:
from pyspark.sql.functions import count, col, count_distinct, when

print("--- RAW LAYER AUDIT ---")
raw_df = spark.read.json(raw_path)

# 1. Basic Volume Check
total_count = raw_df.count()
print(f"Total records in Raw: {total_count}")

# 2. Schema Validation
print("\nSchema Structure:")
raw_df.printSchema()

# 3. Data Quality: Check for Nulls in critical keys
# In GitHub data, 'id' and 'type' should never be null
null_counts = raw_df.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["id", "type", "created_at"]
])

print("\nNull Value Audit (Critical Columns):")
null_counts.show()

# 4. Event Type Distribution
# This helps confirm the partition or file contains the expected data
print("\nTop 5 Event Types in this batch:")
raw_df.groupBy("type").count().orderBy(col("count").desc()).show(5)

# 5. Sample Preview
print("\nRaw Data Sample:")
raw_df.select("id", "type", "actor.login", "repo.name", "created_at").show(5, truncate=False)

--- RAW LAYER AUDIT ---


Total records in Raw: 180387

Schema Structure:
root
 |-- actor: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- display_login: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- org: struct (nullable = true)
 |    |-- avatar_url: string (nullable = true)
 |    |-- gravatar_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- login: string (nullable = true)
 |    |-- url: string (nullable = true)
 |-- payload: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _links: struct (nullable = true)
 |    |    |    |-- html: struct (nullable = true)
 |    |    |    |    |-- href: string (nullable = tru

+---+----+----------+
| id|type|created_at|
+---+----+----------+
|  0|   0|         0|
+---+----+----------+


Top 5 Event Types in this batch:


+-----------------+------+
|             type| count|
+-----------------+------+
|        PushEvent|118979|
|      CreateEvent| 20327|
| PullRequestEvent| 12470|
|       WatchEvent|  7236|
|IssueCommentEvent|  6651|
+-----------------+------+
only showing top 5 rows


Raw Data Sample:
+-----------+----------------+-----------------+--------------------------------------+--------------------+
|id         |type            |login            |name                                  |created_at          |
+-----------+----------------+-----------------+--------------------------------------+--------------------+
|34502641367|MemberEvent     |mujianwu         |Uotan-Dev/UotanWorkStation-ROM-Builder|2024-01-01T15:00:00Z|
|34502641374|PushEvent       |mdmaid69         |mdmaid69/reimagined-giggle            |2024-01-01T15:00:00Z|
|34502641375|ForkEvent       |SynthWave-Systems|ampleforth/ampleforth-contracts       |2024-01-01T15:00:00Z|
|34502641376|PullRequestEvent|tobexyz          |tobexyz/yaac

### Bronze Audit
This shows the initial ingestion where applied the manual schema and added metadata.

In [28]:
print("--- BRONZE LAYER AUDIT ---")
bronze_df = spark.read.parquet(bronze_path)

print(f"Total Bronze Records: {bronze_df.count()}")

bronze_df.select("id", "type", "created_at", "_ingestion_timestamp", "_source_file").show(5, truncate=False)

--- BRONZE LAYER AUDIT ---


Total Bronze Records: 180387


+-----------+----------------+--------------------+--------------------------+---------------------+
|id         |type            |created_at          |_ingestion_timestamp      |_source_file         |
+-----------+----------------+--------------------+--------------------------+---------------------+
|34502641367|MemberEvent     |2024-01-01T15:00:00Z|2026-01-20 23:29:29.227738|2024-01-01-15.json.gz|
|34502641374|PushEvent       |2024-01-01T15:00:00Z|2026-01-20 23:29:29.227738|2024-01-01-15.json.gz|
|34502641375|ForkEvent       |2024-01-01T15:00:00Z|2026-01-20 23:29:29.227738|2024-01-01-15.json.gz|
|34502641376|PullRequestEvent|2024-01-01T15:00:00Z|2026-01-20 23:29:29.227738|2024-01-01-15.json.gz|
|34502641379|PushEvent       |2024-01-01T15:00:00Z|2026-01-20 23:29:29.227738|2024-01-01-15.json.gz|
+-----------+----------------+--------------------+--------------------------+---------------------+
only showing top 5 rows



### Silver Audit (Flattened Data)
Here visualize the effect of the explode function. You will notice the row count increases because one event now represents multiple commits.

In [34]:
print("--- SILVER LAYER AUDIT (Flattened Commits) ---")
silver_df = spark.read.parquet(silver_path)
print(f"Total Silver Commit Records: {silver_df.count()}")
silver_df.select("event_id", "user_name", "repo_name", "commit_message").show(5, truncate=False)

--- SILVER LAYER AUDIT (Flattened Commits) ---


Total Silver Commit Records: 153791
+-----------+----------+--------------------------+---------------------------+
|event_id   |user_name |repo_name                 |commit_message             |
+-----------+----------+--------------------------+---------------------------+
|34502641379|dim12512a |dim12512a/Repo5           |Empty Commit               |
|34502641469|mdmaid69  |mdmaid69/reimagined-giggle|last recover hurt off claim|
|34502641494|wodcoredev|wod-core/website          |Update index.html          |
|34502641539|mmomtchev |mmomtchev/ffmpeg          |reeanable cleanup          |
|34502641539|mmomtchev |mmomtchev/ffmpeg          |skip error testing         |
+-----------+----------+--------------------------+---------------------------+
only showing top 5 rows



### Gold Result (The Leaderboard)

This is the final refined data that answers your business question: "Which repositories have the most developer velocity?".

In [3]:
print("--- GOLD LAYER: REPOSITORY LEADERBOARD ---")
gold_df = spark.read.parquet(gold_path).orderBy(F.desc("total_commits"))
gold_df.show(20, truncate=False)



--- GOLD LAYER: REPOSITORY LEADERBOARD ---


+------------------------------------------------------+-------------+-------------------+----------+
|repo_name                                             |total_commits|unique_contributors|event_date|
+------------------------------------------------------+-------------+-------------------+----------+
|dim12512a/Repo6                                       |3144         |1                  |2024-01-01|
|appref5555ix63/Repo3                                  |3144         |1                  |2024-01-01|
|appref5555ix63/Repo2                                  |3144         |1                  |2024-01-01|
|lu146enza/Repo9                                       |3140         |1                  |2024-01-01|
|dim12512a/Repo5                                       |3137         |1                  |2024-01-01|
|dim12512a/Repo8                                       |3135         |1                  |2024-01-01|
|dim12512a/Repo7                                       |3132         |1           